In [1]:
import os
import subprocess
import shutil
from glob import glob
import nibabel as nib
import numpy as np
import monai
import torch
monai.config.print_config()

MONAI version: 0.9.1
Numpy version: 1.21.5
Pytorch version: 1.11.0+cu113
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 356d2d2f41b473f588899d705bbc682308cee52c
MONAI __file__: /home/valentini/dev/Mousenet/venv/lib/python3.8/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.10
Nibabel version: 3.2.2
scikit-image version: 0.19.2
Pillow version: 9.0.1
Tensorboard version: 2.10.0
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.12.0+cu113
tqdm version: 4.63.0
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.0
pandas version: 1.4.1
einops version: 0.4.1
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [4]:
datasets = ["GIN_Mask"]
for dataset in datasets:
    print(dataset, end='\r')
    mris = sorted(glob(os.path.join('dataset3', dataset, 'MRI', "*_id_affine.nii.gz"))) 
    masks = sorted(glob(os.path.join('dataset3', dataset, 'Mask', "*_id_affine.nii.gz")))    
    labels = sorted(glob(os.path.join('dataset3', dataset, 'Labels', "*_id_affine.nii.gz")))   
    atlas = os.path.join('dataset3', 'Atlas', 'P56_Atlas_128_norm_id.nii.gz')
    atlas_mask = os.path.join('dataset3', 'Atlas', 'P56_Annotation_128_norm_id_mask_dilated.nii.gz')
    print(atlas_mask)
    for i, mri in enumerate(mris):
        print(mri, end='\r')
        command = ['antsRegistration']
        command.extend(['-d', '3'])
        command.extend(['-o', '[<Prefix>,<Output>,<OutputInverse>]'])
        command.extend(['-u', '1'])
        command.extend(['-r', '[<Fixed>,<Moving>,1]'])
        command.extend(['-x', '[<FixedMask>]'])

        command.extend(['-t', 'SyN[0.15,3.0,0.0]'])
        command.extend(['-m', 'CC[<Fixed>,<Moving>,1,3]'])
        command.extend(['-c', '[100x100x100x50,1e-9,15]'])
        command.extend(['-s', '3x2x1x0'])    
        command.extend(['-f', '6x4x2x1']) 

        for j, cmd in enumerate(command):
            cmd = cmd.replace("<Fixed>", atlas)
            cmd = cmd.replace("<Moving>", mri)
            cmd = cmd.replace("<Prefix>", "deformable_")
            cmd = cmd.replace("<Output>", mri.replace("_id_affine.nii", "_id_affine_deformable.nii"))
            cmd = cmd.replace("<OutputInverse>", mri.replace("_id_affine.nii", "_id_affine_deformable_inverse.nii"))
            cmd = cmd.replace("<FixedMask>", atlas_mask)
            cmd = cmd.replace("<MovingMask>", masks[i])
            command[j] = cmd

        # Get .mat (affine), warp (deformable) and warped moving image (to atlas space)
        subprocess.call(command)     
        
        # Move warp .nii.gz and rename to correct name
        shutil.move("deformable_0GenericAffine.mat", mri.replace("_id_affine.nii.gz", "_id_affine.mat"))   
        shutil.move("deformable_1Warp.nii.gz", mri.replace("_id_affine.nii.gz", "_id_affine_warp.nii.gz")) 
        shutil.move("deformable_1InverseWarp.nii.gz", mri.replace("_id_affine.nii.gz", "_id_affine_warp_inverse.nii.gz")) 
        
        # Apply warp to Mask to get DEFORMABLE MASK
        command = ['antsApplyTransforms']
        command.extend(['-d', '3'])
        command.extend(['-i', '<Moving>'])
        command.extend(['-r', '<Fixed>'])
        command.extend(['-o', '<Output>'])
        command.extend(['-n', '<Interpolation>'])
        command.extend(['-t', '<DeformableTransform>'])
        command.extend(['-t', '<AffineTransform>'])

        for j, cmd in enumerate(command):
            cmd = cmd.replace("<Fixed>", atlas)
            cmd = cmd.replace("<Moving>", masks[i])
            cmd = cmd.replace("<Output>", masks[i].replace("_id_affine.nii", "_id_affine_deformable.nii"))            
            cmd = cmd.replace("<Interpolation>", "NearestNeighbor")
            cmd = cmd.replace("<DeformableTransform>", mri.replace("_id_affine.nii.gz", "_id_affine_warp.nii.gz")) 
            cmd = cmd.replace("<AffineTransform>", mri.replace("_id_affine.nii.gz", "_id_affine.mat"))   
            command[j] = cmd
        subprocess.call(command) 

        # Apply warp to Labels to get DEFORMABLE LABELS                          
        command = ['antsApplyTransforms']
        command.extend(['-d', '3'])
        command.extend(['-i', '<Moving>'])
        command.extend(['-r', '<Fixed>'])
        command.extend(['-o', '<Output>'])
        command.extend(['-n', '<Interpolation>'])
        command.extend(['-t', '<DeformableTransform>'])
        command.extend(['-t', '<AffineTransform>'])
        for j, cmd in enumerate(command):
            cmd = cmd.replace("<Fixed>", atlas)
            cmd = cmd.replace("<Moving>", labels[i])
            cmd = cmd.replace("<Output>", labels[i].replace("_id_affine.nii", "_id_affine_deformable.nii"))            
            cmd = cmd.replace("<Interpolation>", "MultiLabel")
            cmd = cmd.replace("<DeformableTransform>", mri.replace("_id_affine.nii.gz", "_id_affine_warp.nii.gz"))  
            cmd = cmd.replace("<AffineTransform>", mri.replace("_id_affine.nii.gz", "_id_affine.mat"))  
            command[j] = cmd
        subprocess.call(command) 

dataset3/Atlas/P56_Annotation_128_norm_id_mask.nii.gz


KeyboardInterrupt: 

In [ ]:
datasets = ['GIN_Mean']
for dataset in datasets:
    print(dataset, end='\r')
    mris = sorted(glob(os.path.join('dataset3', dataset, 'MRI', "*_id.nii.gz"))) 
    masks = sorted(glob(os.path.join('dataset3', dataset, 'Mask', "*_id.nii.gz")))    
    labels = sorted(glob(os.path.join('dataset3', dataset, 'Labels', "*_id.nii.gz")))   
    atlas = os.path.join('dataset3', 'Atlas', 'mean_allreg.nii.gz')    
    for i, mri in enumerate(mris): 
        if not os.path.exists(labels[i].replace("_id.nii", "_id_mean_affine_deformable.nii")):
            # Apply .mat to Labels to get AFFINE LABELS
            if dataset == 'GIN_Mean':            
                command = ['antsApplyTransforms']
                command.extend(['-d', '3'])
                command.extend(['-i', '<Moving>'])
                command.extend(['-r', '<Fixed>'])
                command.extend(['-o', '<Output>'])
                command.extend(['-n', '<Interpolation>'])
                command.extend(['-t', '<AffineTransform>'])

                for j, cmd in enumerate(command):
                    cmd = cmd.replace("<Fixed>", atlas)
                    cmd = cmd.replace("<Moving>", labels[i])
                    cmd = cmd.replace("<Output>", labels[i].replace("_id.nii", "_id_mean_affine.nii"))            
                    cmd = cmd.replace("<Interpolation>", "MultiLabel")
                    cmd = cmd.replace("<AffineTransform>", mri.replace("_id.nii.gz", "_id_mean_affine.mat"))  
                    command[j] = cmd

                subprocess.call(command) 

                # Apply .mat and warp to Labels to get DEFORMABLE LABELS                          
                command = ['antsApplyTransforms']
                command.extend(['-d', '3'])
                command.extend(['-i', '<Moving>'])
                command.extend(['-r', '<Fixed>'])
                command.extend(['-o', '<Output>'])
                command.extend(['-n', '<Interpolation>'])
                command.extend(['-t', '<DeformableTransform>'])
                command.extend(['-t', '<AffineTransform>'])

                for j, cmd in enumerate(command):
                    cmd = cmd.replace("<Fixed>", atlas)
                    cmd = cmd.replace("<Moving>", labels[i])
                    cmd = cmd.replace("<Output>", labels[i].replace("_id.nii", "_id_mean_affine_deformable.nii"))            
                    cmd = cmd.replace("<Interpolation>", "MultiLabel")
                    cmd = cmd.replace("<DeformableTransform>", mri.replace("_id.nii.gz", "_id_mean_affine_warp.nii.gz"))  
                    cmd = cmd.replace("<AffineTransform>", mri.replace("_id.nii.gz", "_id_mean_affine.mat"))  
                    command[j] = cmd

                subprocess.call(command) 

In [ ]:
datasets = ["Feminad_Mean"]
for dataset in datasets:
    print(dataset, end='\r')
    mris = sorted(glob(os.path.join('dataset3', dataset, 'MRI', "*_id.nii.gz"))) 
    masks = sorted(glob(os.path.join('dataset3', dataset, 'Mask', "*_id.nii.gz")))    
    atlas = os.path.join('dataset3', 'Atlas', 'mean_allreg_feminad.nii.gz')
    for i, mri in enumerate(mris): 
        if not os.path.exists(masks[i].replace("_id.nii", "_id_mean_affine_deformable.nii")):
            print(mri, end='\r')
            command = ['antsRegistration']
            command.extend(['-d', '3'])
            command.extend(['-o', '[<Prefix>,<Output>,<OutputInverse>]'])
            command.extend(['-u', '1'])
            command.extend(['-r', '[<Fixed>,<Moving>,1]'])

            command.extend(['-t', 'Rigid[0.1]'])
            command.extend(['-m', 'MI[<Fixed>,<Moving>,1,32,Regular,0.2]'])
            command.extend(['-c', '[2000x2000x2000,1e-9,15]'])
            command.extend(['-s', '2x1x0'])    
            command.extend(['-f', '4x2x1']) 

            command.extend(['-t', 'Affine[0.1]'])
            command.extend(['-m', 'MI[<Fixed>,<Moving>,1,32,Regular,0.1]'])
            command.extend(['-c', '[2000x2000x2000,1e-9,15]'])
            command.extend(['-s', '2x1x0'])    
            command.extend(['-f', '4x2x1']) 

            command.extend(['-t', 'SyN[0.15,3.0,0.0]'])
            command.extend(['-m', 'CC[<Fixed>,<Moving>,1,4]'])
            command.extend(['-c', '[100x100x100x50,1e-9,15]'])
            command.extend(['-s', '3x2x1x0'])    
            command.extend(['-f', '6x4x2x1']) 

            for j, cmd in enumerate(command):
                cmd = cmd.replace("<Fixed>", atlas)
                cmd = cmd.replace("<Moving>", mri)
                cmd = cmd.replace("<Prefix>", "deformable_")
                cmd = cmd.replace("<Output>", mri.replace("_id.nii", "_id_mean_affine_deformable.nii"))
                cmd = cmd.replace("<OutputInverse>", mri.replace("_id.nii", "_id_mean_affine_deformable_inverse.nii"))
                command[j] = cmd

            # Get .mat (affine), warp (deformable) and warped moving image (to atlas space)
            subprocess.call(command)     

            # Move affine .mat and rename to correct name
            shutil.move("deformable_0GenericAffine.mat", mri.replace("_id.nii.gz", "_id_mean_affine.mat"))         
            # Move warp .nii.gz and rename to correct name
            shutil.move("deformable_1Warp.nii.gz", mri.replace("_id.nii.gz", "_id_mean_affine_warp.nii.gz")) 
            shutil.move("deformable_1InverseWarp.nii.gz", mri.replace("_id.nii.gz", "_id_mean_affine_warp_inverse.nii.gz")) 

            # Apply .mat to MRI to get AFFINE MRI
            command = ['antsApplyTransforms']
            command.extend(['-d', '3'])
            command.extend(['-i', '<Moving>'])
            command.extend(['-r', '<Fixed>'])
            command.extend(['-o', '<Output>'])
            command.extend(['-n', '<Interpolation>'])
            command.extend(['-t', '<AffineTransform>'])

            for j, cmd in enumerate(command):
                cmd = cmd.replace("<Fixed>", atlas)
                cmd = cmd.replace("<Moving>", mri)
                cmd = cmd.replace("<Output>", mri.replace("_id.nii", "_id_mean_affine.nii"))            
                cmd = cmd.replace("<Interpolation>", "Linear")
                cmd = cmd.replace("<AffineTransform>", mri.replace("_id.nii.gz", "_id_mean_affine.mat"))  
                command[j] = cmd

            subprocess.call(command) 

            # Apply .mat to Mask to get AFFINE MASK
            command = ['antsApplyTransforms']
            command.extend(['-d', '3'])
            command.extend(['-i', '<Moving>'])
            command.extend(['-r', '<Fixed>'])
            command.extend(['-o', '<Output>'])
            command.extend(['-n', '<Interpolation>'])
            command.extend(['-t', '<AffineTransform>'])

            for j, cmd in enumerate(command):
                cmd = cmd.replace("<Fixed>", atlas)
                cmd = cmd.replace("<Moving>", masks[i])
                cmd = cmd.replace("<Output>", masks[i].replace("_id.nii", "_id_mean_affine.nii"))            
                cmd = cmd.replace("<Interpolation>", "NearestNeighbor")
                cmd = cmd.replace("<AffineTransform>", mri.replace("_id.nii.gz", "_id_mean_affine.mat"))  
                command[j] = cmd

            subprocess.call(command) 

            # Apply .mat and warp to Mask to get DEFORMABLE MASK
            command = ['antsApplyTransforms']
            command.extend(['-d', '3'])
            command.extend(['-i', '<Moving>'])
            command.extend(['-r', '<Fixed>'])
            command.extend(['-o', '<Output>'])
            command.extend(['-n', '<Interpolation>'])
            command.extend(['-t', '<DeformableTransform>'])
            command.extend(['-t', '<AffineTransform>'])

            for j, cmd in enumerate(command):
                cmd = cmd.replace("<Fixed>", atlas)
                cmd = cmd.replace("<Moving>", masks[i])
                cmd = cmd.replace("<Output>", masks[i].replace("_id.nii", "_id_mean_affine_deformable.nii"))            
                cmd = cmd.replace("<Interpolation>", "NearestNeighbor")
                cmd = cmd.replace("<DeformableTransform>", mri.replace("_id.nii.gz", "_id_mean_affine_warp.nii.gz"))  
                cmd = cmd.replace("<AffineTransform>", mri.replace("_id.nii.gz", "_id_mean_affine.mat"))  
                command[j] = cmd

            subprocess.call(command) 

In [ ]:
#new-GIN